In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk

In [ ]:
provider = q.IBMQ.get_provider(group='qiskit-terra', project='pulse-testing')
backend = provider.get_backend('alt_almaden')
#from qiskit.test.mock import FakeAlmaden
#backend = FakeAlmaden()
system = q.pulse.PulseChannelSpec.from_backend(backend)
config = backend.configuration()
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
theta = np.pi/3

In [ ]:
schedules = []

In [ ]:
schedule = (cmd_def.get('u2', qubits=[0], P0=0, P1=1) | cmd_def.get('u2', qubits=[1], P0=-np.pi/2, P1=np.pi/2))

schedule |= cmd_def.get('cx', qubits=[0, 1]) << schedule.duration
schedule |= cmd_def.get('u1', qubits=[1], P0=theta) << schedule.duration
schedule |= cmd_def.get('cx', qubits=[0, 1]) << schedule.duration

schedule |= (cmd_def.get('u2', qubits=[0], P0=np.pi, P1=-np.pi) | cmd_def.get('u2', qubits=[1], P0=-np.pi/2, P1=np.pi/2)) << schedule.duration

schedule |= measure << schedule.duration

schedules.append(schedule)

In [ ]:
N = 2
circ = q.QuantumCircuit(N)
circ.ry(np.pi / 2, 0)
circ.rx(np.pi / 2, 1)
circ.zz_interaction(theta, 0, 1)
circ.ry(-np.pi / 2, 0)
circ.rx(np.pi / 2, 1)

decomposed_circuit = circ.decompose()

pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
transpiled_circuit = q.transpile(decomposed_circuit, backend, optimization_level=1)

schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
schedule |= measure << schedule.duration
schedules.append(schedule)

In [ ]:
schedules[0].draw(table=False, channels=[config.drive(0), config.drive(1), config.control(0)])

In [ ]:
schedules[1].draw(table=False, channels=[config.drive(0), config.drive(1), config.control(0)])

In [ ]:
shots = 8000
job = q.execute(schedules, backend=backend, shots=shots)
print('job is %s' % job.job_id())

In [ ]:
job.status()

# Run a mitigation cal job

In [ ]:
import qiskit.ignis.mitigation.measurement as mit
meas_qcs, meas_labels = mit.complete_meas_cal(qubit_list=[0,1])
meas_qcs_transpiled = q.transpile(meas_qcs, backend, basis_gates=['x'])
meas_schedules = q.schedule(meas_qcs_transpiled, backend=backend, cmd_def=cmd_def)

In [ ]:
job_mit = q.execute(meas_schedules, backend, shots=shots)
print('job_mit is %s' % job_mit.job_id())

In [ ]:
job_mit.status()

-----
## Analysis

In [ ]:
from qiskit.result import marginal_counts

In [ ]:
result = job.result()

In [ ]:
# mitigate results
result_mit = marginal_counts(job_mit.result(), [0,1])
readout_fitter = mit.CompleteMeasFitter(result_mit, meas_labels)
readout_fitter.cal_matrix

In [ ]:
standard_counts = marginal_counts(result.get_counts(0), [0,1])
print('before mitigation:')
print(standard_counts)

standard_counts = readout_fitter.filter.apply(standard_counts)
print('\n\n\n after mitigation:')
print(standard_counts)

In [ ]:
optimized_counts = marginal_counts(result.get_counts(1), [0,1])
print('before mitigation:')
print(optimized_counts)

optimized_counts = readout_fitter.filter.apply(optimized_counts)
print('\n\n\n after mitigation:')
print(optimized_counts)

In [ ]:
# sanitize data and plot
for key in standard_counts:
    standard_counts[key] = int(standard_counts[key])
for key in optimized_counts:
    optimized_counts[key] = int(optimized_counts[key])

In [ ]:
standard_counts

In [ ]:
optimized_counts

In [ ]:
from qiskit.visualization import plot_histogram
plot_histogram([standard_counts, optimized_counts], legend=['standard', 'optimized'])

In [ ]:
ideal_counts = {'01': 0.25, '10': 0.75}  # https://algassert.com/quirk#circuit=%7B%22cols%22%3A%5B%5B%22Y%5E%C2%BD%22%2C%22X%5E%C2%BD%22%5D%2C%5B%22%E2%80%A2%22%2C%22X%22%5D%2C%5B1%2C%7B%22id%22%3A%22Rzft%22%2C%22arg%22%3A%22pi%2F3%22%7D%5D%2C%5B%22%E2%80%A2%22%2C%22X%22%5D%2C%5B%22Y%5E-%C2%BD%22%2C%22X%5E%C2%BD%22%5D%5D%7D
# standard_counts =
# optimized_counts =

In [ ]:
print(pulse_compiler_helper_fns.kl_divergence(ideal_counts, standard_counts))
print(pulse_compiler_helper_fns.kl_divergence(ideal_counts, optimized_counts))

In [ ]:
print(pulse_compiler_helper_fns.cross_entropy(ideal_counts, standard_counts))
print(pulse_compiler_helper_fns.cross_entropy(ideal_counts, optimized_counts))

In [ ]:
from qiskit.quantum_info import hellinger_fidelity
print(hellinger_fidelity(ideal_counts, standard_counts))
print(hellinger_fidelity(ideal_counts, optimized_counts))